

# Variational Fair Information bottlekneck

### Implemented in TensorFlow


This notebook addresses the  topic of engineering fair(-er) data representations, devoid of bias from the sensitive features (e.g. age, sex, race).
Idea is to generate a representation of the original data, removing discriminatory associations while preserving its original quality.

- The implemented architecture is described in http://www.cs.toronto.edu/~sajadn/sajad_norouzi/CSC2541_report.pdf
- The dataset used in this notebook is the Adult dataset, retrieved from the UCI ML repository
- The sensitive feature is given to be the age of a person

In [1]:
# HIDE
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from IPython import display

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn import metrics

import tensorflow as tf
import tensorflow.keras as ke
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Add, Multiply


In [2]:
np.random.seed(7)
warnings.filterwarnings("ignore", category=FutureWarning) 
%matplotlib inline

In [3]:
print(f"numpy: {np.__version__}")
print(f"sklearn: {sk.__version__}")
print(f"pandas: {pd.__version__}")
print(f"TensorFlow: {tf.__version__}")
print(f"keras: {keras.__version__}")

numpy: 1.19.2
sklearn: 0.23.2
pandas: 1.1.3
TensorFlow: 2.4.1
keras: 2.4.0


### Column names and dtypes



In [4]:
# Columns with categorical variables
cat_columns = [
        "Workclass", "Education", "Country", "Relationship",
        "Martial Status", "Occupation", "Relationship",
        "Race", "Sex"
    ]

# All columns
columns = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status", \
              "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
                "Hours per week", "Country", "Target"]

# Data types
types = {0: int, 1: str, 2: int, 3: str, 4: int, 5: str, 6: str, 7: str, 8: str, 9: str, 10: int,
                                11: int, 12: int, 13: str, 14: str}


### Implementing the P-rule
From (Biddle, 2005), lets us verify proportionality in outcomes per sensitive feature value

In [5]:

def p_rule(y_pred, x_sensitive, threshold=0.5):
    y_z_1 = y_pred[x_sensitive == 1] > threshold if threshold else y_pred[x_sensitive == 1]
    y_z_0 = y_pred[x_sensitive == 0] > threshold if threshold else y_pred[x_sensitive == 0]
    odds = y_z_1.mean() / y_z_0.mean()
    return np.min([odds, 1/odds]) * 100


### Pre-processing functions for Adult dataset
- Source of data : https://archive.ics.uci.edu/ml/datasets/adult

In [6]:
# remove observations with missing feature values
def remove_missing(X):
    m = X.shape[0]
    print('Raw Dataset size : ', m)
    X.replace('nan', np.nan, inplace=True)
    X.dropna(inplace=True)
    n = X.shape[0]
    print('Size after dropping null values: ', n)
    print('Removed ', (m-n), ' observations')

    
# one-hot encode categorical data
def replace_categorical(X):
    X_sets = [X.select_dtypes(include=[i]).copy() for i in ['object', 'int']]
    X_cat , X_n = X_sets
    [print(n, ' set size:' , i.shape) 
     for (n,i) in zip(['\nCategorical',
                       'Continuous'],
                          X_sets)]
    
    X_cat = pd.get_dummies(X_cat, columns=cat_columns)
    return pd.concat([X_n, X_cat], axis=1)


# Split features and labels
def separate_label(X):
    y = X['Target'].copy()
    X = X.drop(['Target'], axis=1)
    y = LabelEncoder().fit_transform(y)
    return X, y


# Binarize continuous features with column mean
def binarize_features(X):
    for i in range(6):
        thresh = X.iloc[:, i].mean()
        X.iloc[:, i] = np.where(X.iloc[:, i].values > thresh, 1,0)
    return X


# Load and preprocess
def load_adult(binarize=False):
    data = pd.read_csv(
        "./VFIB/data/adult/adult.csv",
        names=columns,
        sep=r'\s*,\s*',
        engine='python', skiprows=1,
        na_values="?",
        dtype=types)
    
    remove_missing(data)
    X, y = separate_label(data)
    X = replace_categorical(data)
    
    if binarize:
        binarize_features(X)

    return train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)



### Load Adult Dataset
- `binarize` flag to binarize continuous features
- Otherwise scaling required

In [7]:
data = load_adult(binarize=True)

X_train, X_test, y_train, y_test = [i.astype(np.float32) for i in data]

Raw Dataset size :  48842
Size after dropping null values:  45222
Removed  3620  observations

Categorical  set size: (45222, 9)
Continuous  set size: (45222, 6)


### Examine data

In [8]:
_ = [print(i.shape) for i in data]

(31655, 110)
(13567, 110)
(31655,)
(13567,)


In [9]:
X_train.head(3)

,Age,fnlwgt,Education-Num,Capital Gain,Capital Loss,Hours per week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Private,Workclass_Self-emp-inc,...,Relationship_Own-child,Relationship_Unmarried,Relationship_Wife,Race_Amer-Indian-Eskimo,Race_Asian-Pac-Islander,Race_Black,Race_Other,Race_White,Sex_Female,Sex_Male
27314,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
31381,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
30713,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


### Define Encoder Layer
- Variational encoder, with 3 outputs: 
mean (`mu`), log_variance (`sigma`) and latent representation (`z`)  
- `Sampling` class used to reparametrize latent representation `z` 

In [10]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding of an observation."""

    def call(self, inputs):
        z_mu, z_log_sigma = inputs
        batch = tf.shape(z_mu)[0]
        dim = tf.shape(z_mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mu + tf.exp(0.5 * z_log_sigma) * epsilon


class Encoder(keras.layers.Layer):
    def __init__(self, input_dim, latent_dim, act):
        super(Encoder, self).__init__()
        self.dense_1 = Dense(input_dim*2, activation=act, input_shape=(input_dim,))
        self.dense_2 = Dense(input_dim/4, activation=act)
        self.z_mean =  Dense(latent_dim, name='mu')
        self.z_log_sigma = Dense(latent_dim, name='log_sigma')
        
        
    def call(self, x):        
        x = self.dense_1(x)
        x = self.dense_2(x)
        
        mean = self.z_mean(x)
        log_sigma = self.z_log_sigma(x)
        z = Sampling()([mean, log_sigma])
        
        return mean, log_sigma, z

### Examine Encoder output
- Each layer has an output shape of `n x l`, where `n` is number of observations and `l` is size of latent representation

In [11]:
input_dim = X_train.shape[-1]
latent_dim = 2

In [12]:
output_encoder = Encoder(input_dim, latent_dim, 'relu')(X_train.values)
mu, sigma, z = output_encoder

In [13]:
_ = [print(n, 'output shape :', i.shape) for n,i in zip(['mu', 'sigma', 'z'],output_encoder)]

mu output shape : (31655, 2)
sigma output shape : (31655, 2)
z output shape : (31655, 2)


### Define Classifier

In [14]:
class Classifier(keras.layers.Layer):
    def __init__(self, latent_dim, act, n_sens = 0):
        super(Classifier, self).__init__()
        self.dense_1 = Dense(64, activation=act, input_shape=(latent_dim+n_sens,))
        self.dense_2 = Dense(32, activation=act)
        self.dropout =  Dropout(0.2)
        self.output_layer = Dense(1)
        
    def call(self, x):
        x = self.dense_1(x)
        x = self.dropout(x)
        x = self.dense_2(x)
        x = self.dropout(x)
        return self.output_layer(x)

### Examine classifier output
- `n x 1`, where n is number of training observations

In [15]:
sensitive_attr = X_train.Age.values.reshape(-1,1)
n_sens = 1
output_predictor = Classifier(latent_dim, 'relu', n_sens=1)(tf.concat([mu, sensitive_attr], axis=1))
output_predictor

<tf.Tensor: shape=(31655, 1), dtype=float32, numpy=
array([[-0.0793968 ],
       [-0.09409514],
       [-0.00398894],
       ...,
       [ 0.00334169],
       [-0.00185159],
       [ 0.00075468]], dtype=float32)>

### Define VFIB model
- Implemented with 3 different losses:
    * `Negative log bernoulli` for classification
    * `Kullback Leiber Divergence` constrains latent distribution to original data distribution
    * `Maximum mean discrepency` forces model to match moments between marginal posterior distributions of latent variables w.r.t sensitive attribute values; i.e., qφ(z|s = 0) and qφ(z|s = 1)
    


In [19]:
class VFIB(keras.Model):
    def __init__(self, encoder, predictor, feature_dim,loss_type,  **kwargs):
        super(VFIB, self).__init__(**kwargs)
        self.encoder = encoder
        self.classifier = predictor
        self.loss_type = loss_type
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.prediction_loss_tracker = keras.metrics.Mean(
            name="prediction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.mmd_loss_tracker = keras.metrics.Mean(name="mmd_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.prediction_loss_tracker,
            self.kl_loss_tracker,
            self.mmd_loss_tracker
        ]
    
    @tf.function
    def neg_log_bernoulli(self, true, pred, mean=True, clamp=True):
        if clamp:
            pred = K.clip(pred, -9.5, 9.5)
            
        batch_size = K.shape(true)[0]

        mdata = tf.reshape( true, (batch_size,1) )
        mmu = tf.reshape( pred, (batch_size,1) )

        log_prob_1 = tf.math.log_sigmoid(mmu)
        log_prob_2 = tf.math.log_sigmoid(-mmu)
        return -tf.reduce_mean((mdata*log_prob_1)+(1-mdata)*log_prob_2)
    
    @tf.function
    def KL(self, mu, log_sigma):
        kl_loss = 0.5 * tf.reduce_mean(( - log_sigma + K.square(mu) + K.exp(log_sigma)))
        return kl_loss
    

    @tf.function
    def mmd_loss(self, X, z):
        
        def md(t, l):
            s_0 = tf.where(t[:,0]==0)
            s_1 =tf.where(t[:,0]==1)

            z_0 = tf.gather(l, s_0)
            z_1 = tf.gather(l, s_1)

            z_0 = tf.reshape(z_0, (K.shape(z_0)[0], K.shape(z_0)[-1]))
            z_1 = tf.reshape(z_1, (K.shape(z_1)[0], K.shape(z_1)[-1]))
            return z_0, z_1
        
        def kernel(a,b):
            dist1 = tf.expand_dims(tf.math.reduce_sum((a**2), axis=1), axis=1) * tf.ones(shape=(1,K.shape(b)[0]))
            dist2 = tf.expand_dims(tf.math.reduce_sum((b**2), axis=1), axis=0)* tf.ones(shape=(K.shape(a)[0], 1))
            dist3 = tf.matmul(a, tf.transpose(b, perm=[1, 0]))
            dist = (dist1 + dist2) - (2 * dist3)
            return tf.reduce_mean(tf.math.exp(-dist))
        
        z_s_0, z_s_1 = md(X, z)
        loss = kernel(z_s_0, z_s_0) + kernel(z_s_1, z_s_1) - 2 * kernel(z_s_0, z_s_1)
        return loss
    
    def split_sensitive_X(self, tensor, col, n):
        '''takes Xn (2D feature tensor) and returns 2 tensors(sensitive features and normal features)'''
        dim = tensor.shape[-1]
        pre, sens, post =  tf.split(tensor, (col, n, (dim-(col+n))), axis=1)
        return sens, tf.concat([pre, post], axis=1)
    
    def call(self, inputs):
        sens, _ = self.split_sensitive_X(inputs, 0, 1)
        mu, sig, z = self.encoder(inputs)
        return self.classifier(tf.concat([z, sens], 1))
        
        
    def train_step(self, data):
        X, y = data
        with tf.GradientTape() as tape:
            
            z_mean, z_log_sigma, z = self.encoder(X)
            
            sens, _ = self.split_sensitive_X(X, 0, 1)

            preds = self.classifier(tf.concat([z, sens], axis=1))

            prediction_loss = self.neg_log_bernoulli(y, preds)

            kl_loss = self.KL(z_mean, z_log_sigma)
            
            mmd_loss = self.mmd_loss(X, z_mean)
            
            if self.loss_type=='all':
                total_loss =  prediction_loss+ kl_loss + mmd_loss
            elif self.loss_type=='kl':
                total_loss =  prediction_loss+ kl_loss
            else:
                total_loss =  prediction_loss
                
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.prediction_loss_tracker.update_state(prediction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.mmd_loss_tracker.update_state(mmd_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "classification_loss": self.prediction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "mmd_loss": self.mmd_loss_tracker.result()
        }


### Training period
The model is trained for `100 epochs`, repeated 10 times (i.e. 10 training periods).
All other hyperparameters are kept the same as the original implementation.


In [25]:
def training_period(epochs, bs):
    encoding_module = Encoder(input_dim, latent_dim, act)
    prediction_module = Classifier(latent_dim, act)
    model = VFIB(encoding_module, prediction_module, input_dim, loss_type)
    model.compile(optimizer=opt)
    model.fit(X_train, y_train, batch_size=bs, epochs=epochs)
    y_pred = model.predict(X_test.values)
    y_pred_bin = np.where(y_pred>0.5,1,0)
    return y_pred, y_pred_bin

### Compairason of model performance with different losses

- Losses compared:
    * `NLB` + `KLD` + `MMD`
    * `NLB` + `KLD`
    * `NLB`
- Accuracy of models evaluated using `roc` and `accuracy` score
- Fairness evaluated using `p-rule`

In [ ]:

latent_dim=50
act = 'tanh'
opt = tf.keras.optimizers.Adam(lr=0.002)
training_periods = 10

In [27]:
losses = {}
for loss_type in ['all','kld','nlb']:
    res = {}
    for i in range(training_periods):
        print('Run %d for %s'%(i, loss_type))  
        y_pred, y_pred_bin = training_period(epochs=100, bs=128 )
        if i == 0:            
            res['roc']=[metrics.roc_auc_score(y_test,y_pred)]
            res['acc']= [metrics.accuracy_score(y_test,y_pred_bin)]
            res['p-rule']=[p_rule(y_pred_bin, X_test.Age)]
        else:
            res['roc']+=[metrics.roc_auc_score(y_test, y_pred)]
            res['acc']+= [metrics.accuracy_score(y_test,y_pred_bin)]
            res['p-rule']+=[p_rule(y_pred_bin, X_test.Age)]
    losses[loss_type]=res


Run 0 for all
Epoch 1/100


C:\Users\npurk\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_3:1", shape=(None,), dtype=int64), values=Tensor("PartitionedCall_3:0", shape=(None, 50), dtype=float32), dense_shape=Tensor("PartitionedCall_3:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


248/248 [==============================] - 5s 4ms/step - loss: 1.0428 - classification_loss: 0.4394 - kl_loss: 0.5552 - mmd_loss: 0.0383
Epoch 2/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9649 - classification_loss: 0.3943 - kl_loss: 0.5359 - mmd_loss: 0.0283
Epoch 3/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9199 - classification_loss: 0.3665 - kl_loss: 0.5301 - mmd_loss: 0.0221
Epoch 4/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9121 - classification_loss: 0.3630 - kl_loss: 0.5295 - mmd_loss: 0.0183
Epoch 5/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9006 - classification_loss: 0.3551 - kl_loss: 0.5292 - mmd_loss: 0.0168
Epoch 6/100
248/248 [==============================] - 1s 5ms/step - loss: 0.9022 - classification_loss: 0.3577 - kl_loss: 0.5288 - mmd_loss: 0.0156
Epoch 7/100
248/248 [==============================] - 1s 5ms/step - loss: 0.9045 - classification_loss: 0.3579 - kl_l

C:\Users\npurk\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_3:1", shape=(None,), dtype=int64), values=Tensor("PartitionedCall_3:0", shape=(None, 50), dtype=float32), dense_shape=Tensor("PartitionedCall_3:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


248/248 [==============================] - 5s 4ms/step - loss: 1.0433 - classification_loss: 0.4313 - kl_loss: 0.5541 - mmd_loss: 0.0344
Epoch 2/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9310 - classification_loss: 0.3729 - kl_loss: 0.5329 - mmd_loss: 0.0214
Epoch 3/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9095 - classification_loss: 0.3598 - kl_loss: 0.5299 - mmd_loss: 0.0185
Epoch 4/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9091 - classification_loss: 0.3626 - kl_loss: 0.5298 - mmd_loss: 0.0174
Epoch 5/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9002 - classification_loss: 0.3571 - kl_loss: 0.5298 - mmd_loss: 0.0139
Epoch 6/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9069 - classification_loss: 0.3591 - kl_loss: 0.5302 - mmd_loss: 0.0156
Epoch 7/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9049 - classification_loss: 0.3581 - kl_l

...

248/248 [==============================] - 5s 4ms/step - loss: 1.0429 - classification_loss: 0.4283 - kl_loss: 0.5537 - mmd_loss: 0.0356
Epoch 2/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9239 - classification_loss: 0.3714 - kl_loss: 0.5306 - mmd_loss: 0.0202
Epoch 3/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9076 - classification_loss: 0.3602 - kl_loss: 0.5299 - mmd_loss: 0.0173
Epoch 4/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9060 - classification_loss: 0.3594 - kl_loss: 0.5302 - mmd_loss: 0.0170
Epoch 5/100
248/248 [==============================] - 1s 4ms/step - loss: 0.8986 - classification_loss: 0.3515 - kl_loss: 0.5298 - mmd_loss: 0.0169
Epoch 6/100
248/248 [==============================] - 1s 4ms/step - loss: 0.8955 - classification_loss: 0.3505 - kl_loss: 0.5303 - mmd_loss: 0.0150
Epoch 7/100
248/248 [==============================] - 1s 4ms/step - loss: 0.8996 - classification_loss: 0.3524 - kl_l

C:\Users\npurk\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_3:1", shape=(None,), dtype=int64), values=Tensor("PartitionedCall_3:0", shape=(None, 50), dtype=float32), dense_shape=Tensor("PartitionedCall_3:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


248/248 [==============================] - 4s 4ms/step - loss: 1.0473 - classification_loss: 0.4333 - kl_loss: 0.5551 - mmd_loss: 0.0357
Epoch 2/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9306 - classification_loss: 0.3762 - kl_loss: 0.5310 - mmd_loss: 0.0204
Epoch 3/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9168 - classification_loss: 0.3708 - kl_loss: 0.5287 - mmd_loss: 0.0174
Epoch 4/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9135 - classification_loss: 0.3660 - kl_loss: 0.5293 - mmd_loss: 0.0188
Epoch 5/100
248/248 [==============================] - 1s 4ms/step - loss: 0.8985 - classification_loss: 0.3535 - kl_loss: 0.5294 - mmd_loss: 0.0165
Epoch 6/100
248/248 [==============================] - 1s 4ms/step - loss: 0.8994 - classification_loss: 0.3527 - kl_loss: 0.5295 - mmd_loss: 0.0159
Epoch 7/100
248/248 [==============================] - 1s 4ms/step - loss: 0.8986 - classification_loss: 0.3537 - kl_l

C:\Users\npurk\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_3:1", shape=(None,), dtype=int64), values=Tensor("PartitionedCall_3:0", shape=(None, 50), dtype=float32), dense_shape=Tensor("PartitionedCall_3:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


248/248 [==============================] - 5s 4ms/step - loss: 1.0436 - classification_loss: 0.4340 - kl_loss: 0.5536 - mmd_loss: 0.0351
Epoch 2/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9362 - classification_loss: 0.3801 - kl_loss: 0.5309 - mmd_loss: 0.0235
Epoch 3/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9192 - classification_loss: 0.3684 - kl_loss: 0.5300 - mmd_loss: 0.0203
Epoch 4/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9121 - classification_loss: 0.3653 - kl_loss: 0.5297 - mmd_loss: 0.0181
Epoch 5/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9056 - classification_loss: 0.3591 - kl_loss: 0.5290 - mmd_loss: 0.0159
Epoch 6/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9058 - classification_loss: 0.3607 - kl_loss: 0.5296 - mmd_loss: 0.0160
Epoch 7/100
248/248 [==============================] - 1s 4ms/step - loss: 0.9033 - classification_loss: 0.3587 - kl_l

### Examining results

Examining the predictions from VFIB using the `p-rule`, we observe that the model converges to more fair latent representations when using all three losses (`NLB`, `KLD`, and `MMD`), while mantaining an `roc` score above 86%

- We see that the p-rule gives a value of 36.5% when model is trained with all losses, 31.6% when trained with `NLB` and `KL`, and finally 33.3% when only trained with `NLB`.

- We note that model accuracy suffers slightly as we add the MMD and KL constraints to the prediction loss


In [28]:
for loss_type in list(losses.keys()):
    print('With loss : %s'%loss_type)
    for r in list(res.keys()):
        print(r, np.mean(losses[loss_type][r]))
    print()


With loss : all
roc 0.8668416745824228
acc 0.8196432520085501
p-rule 36.53695173796427

With loss : kld
roc 0.8744037548518314
acc 0.8239846686813591
p-rule 31.653949099163377

With loss : nlb
roc 0.8750491194638407
acc 0.8224441659910076
p-rule 33.3742886661309



### Note on statistical parity in data
Using the p-rule to plot true target distributions w.r.t the sensitive attribute `age`, we obtain a p-ratio of `~42%`, to which the VFIB comes close `(36.5%)`

In [23]:
# Train Data
p_rule(y_train, X_train.values[:,0])

41.969948364324

In [24]:
# Test Data
p_rule(y_test, X_test.values[:,0])

42.746847811319334